In [1]:
import os
import sys
import datetime as dt
import pandas as pd
import numpy as np
import gurobipy as gp   #Import library as gp in order to know which functions are from gurobi
from gurobipy import GRB
import matplotlib.pyplot as plt
import httpx
from pydantic import parse_obj_as
from fastapi.encoders import jsonable_encoder
from agricore_sp_models.simulation_models import *
from agricore_sp_models.agricore_sp_models import *
from agricore_sp_models.logging import configure_orm_logger
from agricore_sp_models.logging import configure_orm_logger
from typing import List
from collections import defaultdict
from loguru import logger
from contextlib import nullcontext
from loguru import logger
from contextlib import nullcontext
import math


from settings import settings

In [2]:
def test_production_data(populationId, yearNumber):
    try:
        url = f'{settings.ORM_ENDPOINT}/population/{populationId}/farms/get/simulationdata/longperiod'
        params = {'year': yearNumber}
        client = httpx.Client()
        response = client.get(url, params=params, timeout=None)
        if response.status_code == 200:
            response_json = response.json()
            response_object = parse_obj_as(DataToLPDTO, response_json)
            
            return response_object, yearNumber
        
    except Exception as e:
        print(e)
        return None

In [3]:
response_object, yearNumber = test_production_data(930, 2020)

In [ ]:
response_object.values[0].agentSubsidies

In [ ]:
response_object.agriculturalProductions[0].__dict__

In [ ]:
response_object.policyGroupRelations

In [19]:
def calculate_land_production(productions):
   
    # Extract Agents total ha
    # Calculate Agent average production (€) and costs (€) for its productions --> Use later to calculate the GFI
    agents_ha = defaultdict(float)
    agents_prod = defaultdict(lambda: {'prod_profit': 0, 'prod_costs': 0, 'land_value': 0, 'cultivated_area': 0, 'prod_count': 0})
    agents_avg_prod_values = defaultdict(lambda: {'prod_per_land_ha': 0, 'costs_per_land_ha': 0, 'prod_per_land_value': 0, 'costs_per_land_value': 0})

    for prod in productions:
        farm_id = prod.farmId    
        
        agents_ha[farm_id] += prod.cultivatedArea
        agents_prod[farm_id]['prod_profit'] += prod.valueSales 
        # agents_prod[farm_id]['prod_profit'] += prod.cropProduction #TODO: Check if cropProduction is € or tons. If €, use this.
        agents_prod[farm_id]['prod_costs'] += prod.variableCosts
        agents_prod[farm_id]['land_value'] += prod.landValue
        agents_prod[farm_id]['cultivated_area'] += prod.cultivatedArea  
        agents_prod[farm_id]['prod_count'] += 1
        
    for group, data in agents_prod.items():

        prod_per_land_ha = 0
        costs_per_land_ha = 0 

        prod_per_land_value = 0
        costs_per_land_value = 0

        if data['land_value'] != 0 and data['cultivated_area'] != 0:
            prod_per_land_ha = (data['prod_profit'] / data['prod_count']) / (data['cultivated_area'] / data['prod_count'])
            costs_per_land_ha = (data['prod_costs'] / data['prod_count']) / (data['cultivated_area'] / data['prod_count'])

            prod_per_land_value = (data['prod_profit'] / data['prod_count']) / (data['land_value'] / data['prod_count'])
            costs_per_land_value = (data['prod_costs'] / data['prod_count']) / (data['land_value'] / data['prod_count'])
        
        agents_avg_prod_values[group] = {
                'prod_per_land_ha' : prod_per_land_ha,
                'costs_per_land_ha' : costs_per_land_ha,
                'prod_per_land_value' : prod_per_land_value,
                'costs_per_land_value' : costs_per_land_value
            }        
        
    # Average values of profit and costs to set as default values for agents without land
    prod_per_land_sum = 0
    costs_per_land_sum = 0

    prod_per_ha_sum = 0
    costs_per_ha_sum = 0
    
    for group in agents_avg_prod_values.keys():
        prod_per_land_sum += agents_avg_prod_values[group]["prod_per_land_value"]
        costs_per_land_sum += agents_avg_prod_values[group]["costs_per_land_value"]

        prod_per_ha_sum += agents_avg_prod_values[group]["prod_per_land_ha"]
        costs_per_ha_sum += agents_avg_prod_values[group]["costs_per_land_ha"]

    avg_profit = prod_per_land_sum / len(agents_avg_prod_values)
    avg_costs = costs_per_land_sum / len(agents_avg_prod_values)

    avg_profit_ha = prod_per_ha_sum / len(agents_avg_prod_values)
    avg_costs_ha = costs_per_ha_sum / len(agents_avg_prod_values)

    # Set default values to the avg, in case there is an agent witout productions.
    agents_avg_prod_values.default_factory = lambda: {'prod_per_land_ha': avg_profit_ha, 'costs_per_land_ha': avg_costs_ha, 'prod_per_land_value': avg_profit, 'costs_per_land_value': avg_costs, 'prod_count': 0}

    return agents_avg_prod_values, agents_prod, agents_ha


def holder_retire_ev(holder_age, successor_age, holder_suc):
    """
    OUT: Holder_retires = 0 (no), 1 (yes - with successor), 1 (yes - no successor)
    """

    holder_retires = 0

    if holder_age > 64:

        if holder_suc != 0 and successor_age >= 18:
            holder_retires = 1

        else:
            holder_retires = 2

    return holder_retires

In [20]:
def test_agent(input: DataToLPDTO, yearNumber:int, fail_ID, simulationRunId = 0):

    # Read Inputs
    agents:List[ValueToLPDTO] = input.values
    productions:List[AgriculturalProductionDTO] = input.agriculturalProductions
    year:int = yearNumber
    LT_ignore:bool = input.ignoreLP
    LMM_ignore:bool = input.ignoreLMM   # Indicates if the LMM has to be executed or not - If True, skip to Phase 3 

    result_df = pd.DataFrame()

    # Production and land values
    agents_avg_prod_values, agents_prod, agents_ha = calculate_land_production(productions)
    ha_va = [prod.landValue/prod.cultivatedArea if prod.cultivatedArea != 0 else 0 for prod in response_object.agriculturalProductions]
    for i in range(len(ha_va)):
        if np.isnan(ha_va[i]):
            ha_va[i] = 0
    average_ha_va = np.mean(ha_va)


    if fail_ID == 0:
        agent = agents[np.random.randint(100)]
    else:
        for agent_aux in agents:
            if agent_aux.farmId == fail_ID:
                agent = agent_aux
                break

    # Prepare inputs for the algorithm
    agent_total_ha = agents_ha[agent.farmId]
    agent_avg_prod = agents_avg_prod_values[agent.farmId]["prod_per_land_value"]
    agent_avg_costs = agents_avg_prod_values[agent.farmId]["costs_per_land_value"]

    if agent.a_0 == 0 and agent_total_ha != 0:
        agent.a_0 = agents_prod[agent.farmId]['land_value']

    subsidies_nc = 0
    for agentSubsidies in agent.agentSubsidies:
        subsidies_nc += agentSubsidies.value

    # -- Check agent initial state
    # # Check deposits/TA
    # if agent.sE465 == 0:
    #     agent.sE465 += agent.sE420 + agent.sE410

    # # Check equity
    # equity = agent.sE465 - agent.sE490 
    # if equity < 0:
    #     agent.sE465 += agent.sE420 + agent.sE410

    agent.agentHolder.holderSuccessorsAge

    algorithm_inputs = {
        "SE465" : agent.sE465,
        "A_0" : agent.a_0, 
        "SE490" : agent.sE490,
        "SE420" : agent.sE420,
        "SE410" : agent.sE410, 
        "agents_id" : agent.farmId, 
        "holder_age" : agent.agentHolder.holderAge, 
        "holder_succssesor" : agent.agentHolder.holderSuccessors,
        "holder_succssesor_age" : agent.agentHolder.holderSuccessorsAge,
        "land_avg_prod" : agent_avg_prod, 
        "land_avg_costs" : agent_avg_costs, 
        "NFM" : agent.agentHolder.holderFamilyMembers, 
        "SBT_nc" : subsidies_nc, 
        "average_ha_va" : average_ha_va, 
        "AversionRisk" : agent.aversionRiskFactor, 
        "land_ha" : agent_total_ha, 
        "year" : year  
    }        

    return algorithm_inputs
    


In [21]:
def list_check_input(input):
    if isinstance(input, list):
        pass
    else:
        input = [input]
    return input

In [ ]:
inputs = test_agent(response_object, yearNumber, fail_ID = 652736)
inputs

In [104]:
SE465 = inputs["SE465"]
A_0 = inputs["A_0"]
SE490 = inputs["SE490"]
SE420 = inputs["SE420"]
SE410 = inputs["SE410"]
agents_id = inputs["agents_id"]
land_ha = inputs["land_ha"]
land_avg_prod = inputs["land_avg_prod"]
land_avg_costs = inputs["land_avg_costs"]
holder_age = inputs["holder_age"]
holder_suc = inputs['holder_succssesor']
holder_suc_age = inputs['holder_succssesor_age']

year = inputs["year"]
SBT_nc = inputs["SBT_nc"]
average_ha_va = inputs["average_ha_va"]


AversionRisk = 0.5
SBT_c = 0
sim_period = 10
LP_second_phase = False
LMM_transactions = None

nr_agents = 1

##------------ PARAMETERS ------------##
sim_period = sim_period + 1             # Simulation period
i = sim_period*2               
shape = [sim_period, nr_agents]# Shape of the final matrixes
 

##------------ CONSTANTS ------------##
VAT = 5000
NFM = inputs["NFM"]
EFT = 5000

R = 0.005       #Interest
MEPI = 5000      #Multiple Effects Public Income (MEPI) indicator - minimum annual living income established by the national government for year 
ML = 20         #Average maturity of the loans
EPS = 0.05       #SRobj
COV = 0.15       #Farming overhead costs
X = np.array([*range(0, i + 1)])
C_ECON = 10     #Medium length of the economical cycle
PYLD = 0.15 + 0.05 * np.sin(2 * np.pi * X / (C_ECON))   #Average ratio of value of production per value of land


A_va_average = average_ha_va # €/ha

## ---- Matrix to save results
a_f = np.zeros(shape)          # Matrix to save final values of A for all agents
lt_f = np.zeros(shape)         # Matrix to save final values of LT for all agents
ca_f = np.zeros(shape)         # Matrix to save final values of CA for all agents
fa_f = np.zeros(shape)         # Matrix to save final values of FA for all agents
d_f = np.zeros(shape)          # Matrix to save final values of D for all agents
lr_f = np.zeros(shape)         # Matrix to save final values of LR for all agents
e_f = np.zeros(shape)          # Matrix to save final values of E for all agents
sr_f = np.zeros(shape)         # Matrix to save final values of SR for all agents
fni_f = np.zeros(shape)        # Matrix to save final values of FNI for all agents
gfi_f = np.zeros(shape)        # Matrix to save final values of GFI for all agents

a_ha_f = np.zeros(shape)       # Matrix to save final values of Land size (ha) for all agents
a_va_f = np.zeros(shape)       # Matrix to save final values of Land value (€/ha) for all agents
prod_ha_f = np.zeros(shape)    # Matrix to save final values of Land production (€/ha) for all agents
u1_ha_f = np.zeros(shape)      # Matrix to save final values of land, in ha, for sell/buy

u1_f = np.zeros(shape)  
u2_f = np.zeros(shape)   
np_f = np.zeros(shape)         # Matrix to save final values of NP for all agents
year_f = np.zeros(shape)


#Check if inputs are list, and if not list them (i.e. if there is only one agent). To be deleted/adapted later depending on the input formats
SE465 = list_check_input(SE465)
A_0 = list_check_input(A_0)
SE490 = list_check_input(SE490)
SE420 = list_check_input(SE420)
SE410 = list_check_input(SE410)
agents_id = list_check_input(agents_id)
land_ha = list_check_input(land_ha)

In [ ]:
##------------ MODEL Iniciatilization ------------##
model = None
model = gp.Model("Financial Model")

# Declaring variables #
# decision variables (first decisions initialised with zero value)
u1 = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Decision variable 1 - Buy/sell land")

u2 = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Decision variable 2 - Loans")

ca = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float(0), name = "Current Assets")

fa = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Fixed Assets")

lt = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Long term liabilities")

d = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float(0), name = "Deposits")

a = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float(0), name = "Owned Land")

fni = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Farm Net Income")

gfi = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Gross Farm Income")

e = model.addVars(i + 1, nr_agents, vtype = GRB.CONTINUOUS, lb=float("-inf"), name = "Equity")

# # Liquidity ratio
lr = np.zeros((i + 1, nr_agents))

# Solvency ratio
sr = np.zeros((i + 1, nr_agents))

netp = np.zeros((i + 1, nr_agents))

# Land value and production
A_ha = np.zeros((i + 1, nr_agents))     # Land size (ha)
A_va = np.zeros((i + 1, nr_agents))     # Land ha value (€/ha)
prod_ha = np.zeros((i + 1, nr_agents))  # Land production (€/ha)
u1_ha = np.zeros((i + 1, nr_agents))    # Sell/buy land (ha)

# Initialise variables #
for agent_j in range(nr_agents):
    u1[0, agent_j] = 0
    u2[0, agent_j] = 0

    ca[0, agent_j] = SE465[agent_j]
    d[0, agent_j] = ca[0, agent_j]
    fa[0, agent_j] = 0 if np.isnan(A_0[agent_j]) else A_0[agent_j]
    lt[0, agent_j] = SE490[agent_j]
    a[0, agent_j] = 0 if np.isnan(A_0[agent_j] ) else A_0[agent_j] 
    fni[0, agent_j] = SE420[agent_j]
    gfi[0, agent_j] = SE410[agent_j]
    e[0, agent_j] = fa[0, agent_j] + ca[0, agent_j] - lt[0, agent_j]
    lr[0, agent_j] = ca[0, agent_j] / (1 / ML) * lt[0, agent_j]

    if lt[0, agent_j] != 0:
        sr[0, agent_j] = (lt[0, agent_j]) / (fa[0, agent_j] + ca[0, agent_j] - lt[0, agent_j])
    else:
        sr[0, agent_j] = 0

##------------ MODEL UPTADE ------------##
model.update()
model

In [ ]:
# Optimization process #

# -- Simulate each year 
for c_year in range(1, sim_period):
    y = [0 for _ in range(nr_agents)]
    y1 = [0 for _ in range(nr_agents)]

    for pred_year in range(c_year, c_year + sim_period + 1):
        for agent_j in range(nr_agents):
            # ---- Equation ---- #
            a[pred_year, agent_j] = a[pred_year - 1, agent_j] + u1[pred_year, agent_j] 
            fa[pred_year, agent_j] = a[pred_year, agent_j]                    # Total fixed assets

            lt[pred_year, agent_j] = (1 - 1 / ML) * lt[pred_year - 1, agent_j] + u2[pred_year, agent_j]     # Long term liabilities

            gfi[pred_year, agent_j] = PYLD[pred_year] * a[pred_year, agent_j]               # Gross Farm Income 
            
            fni[pred_year, agent_j] = (
                gfi[pred_year, agent_j]
                + SBT_c * a[pred_year, agent_j]
                + SBT_nc
                - 0.1 * a[pred_year, agent_j]
                - VAT
                - EFT
                - 1 / ML * lt[pred_year - 1, agent_j]
                - u1[pred_year, agent_j]
            )

            d[pred_year, agent_j] = (1 + R) * d[pred_year - 1, agent_j] + fni[pred_year - 1, agent_j] - NFM * MEPI + u2[pred_year, agent_j]

            # Currrent assets
            ca[pred_year, agent_j] = d[pred_year, agent_j]

            # Equity   
            e[pred_year, agent_j] = fa[pred_year, agent_j] + ca[pred_year, agent_j] - lt[pred_year, agent_j]

            # ---- Inequation - Constraints ---- #
            model.addConstr(d[pred_year, agent_j] >= 0, name = "Deposits >= 0")
                                                        
            # ---- Parameters limits - Constraints ---- #                   
            model.addConstr(a[pred_year, agent_j] >= 0, name = "Land >= 0")
                
            # -- Loan adquisition
            model.addConstr(u2[pred_year, agent_j] >= 0, name = "Loans >= 0")                       # Loans cannot be negative
            model.addConstr(u2[pred_year, agent_j] <= 0.95 * e[pred_year - 1, agent_j], name = "Loans <= x Equity")            # Loans adquisition limitation

            # -- Land buy/sell expectations
            model.addConstr(u1[pred_year, agent_j] <= d[pred_year - 1, agent_j], name = "U1 (buy) <= deposits")      # It is not possible to invest in land buy above the deposits capacity    
            model.addConstr(u1[pred_year, agent_j] >= -a[pred_year - 1, agent_j], name = "U1 (sell) <= land")     # It is not possible to sell more land than available
            model.addConstr(u1[pred_year, agent_j] <= 1.5 * u2[pred_year, agent_j], name = "u1 (buy) <= 1.5 u2") 

            # ---- Objective Functions ---- #
            alfa = AversionRisk ** (pred_year - c_year)      #Discount factor - high AversionRisk (riskAversion) higer importance to the future

            # y[agent_j] = y[agent_j] + fni[pred_year, agent_j] / e[c_year - 1, agent_j]    # - Case 1
            y[agent_j] = y[agent_j] + alfa * fa[pred_year, agent_j] + ca[pred_year, agent_j]       # - Case 2
            y1[agent_j] = y1[agent_j] + lt[pred_year, agent_j]
                   

    model.Params.OutputFlag = 0  # Set to 0 to suppress output during optimization

    # Evaluate holder retirment

    holder_retire = holder_retire_ev(holder_age, holder_suc_age, holder_suc) 
    if holder_retire == 2:
        """ 
            If the agent aims to retire and has no successor 
            the optimisation is simplifed to 1 year (no prediciton) 
            and objective funcion aims to sell everything and minimise loans 
        """
        y[agent_j] = ca[c_year, agent_j] - lt[c_year, agent_j]
        model.setObjective(y[0], GRB.MAXIMIZE)
        
    else:
         # REMARK ---> model.setObjectiveN MINIMISE | Priority works inverse (higher number -> higher priority) | Weight works only if same priority
        # - Case 1 - Max FNI/e
        # model.setObjectiveN(-y[0], index=0, priority=1, weight=0.9)
        # model.setObjectiveN(y1[0], index=1, priority=1, weight=0.1)

        # - Case 2 - Max TA
        model.setObjectiveN(-y[0], index=0, priority=1, weight=0.5)
        model.setObjectiveN(y1[0], index=1, priority=1, weight=0.6)

    model.update()
    model.optimize()

    # Update holder age
    holder_age += 1
    holder_suc_age += 1

    # Evaluate if the optimisation is feasible
    # model.status == GRB.INFEASIBLE
    if model.Status == 4 or model.Status == 3 or model.Status == 5:
        print(f"Model {agents_id[agent_j]} is infeasible. Computing IIS...")
        model.computeIIS()
        model.write("model.ilp")

        print("\nThe following constraints are in the IIS:")
        for c in model.getConstrs():
            if c.IISConstr:
                print(f"{c.constrName}")

        # print("Model INFEASIBLE. Attempting feasibility relaxation...")
        # relaxobjtype = 2  # 0: Minimize the weighted sum of relaxations. | 1: Minimize the weighted sum of squared relaxations. | 2: Minimize the weighted maximum of relaxations.
        # minrelax = True   # Minimize the relaxation
        # lbpen = False      # Allow relaxation of lower bounds
        # ubpen = True      # Allow relaxation of upper bounds
        # rhspen = True     # Allow relaxation of RHS of constraints
    
        model.feasRelaxS(0, True, False, True)
        model.optimize()

    if model.status == GRB.INFEASIBLE:
        print(f"Model {agents_id[agent_j]} is still infeasible.")
        break

    else:
        print("Model is feasible")
        # ---- Extract results ---- #
        for agent_j in range(nr_agents):
            a[c_year, agent_j] = a[c_year, agent_j].getValue()
            lt[c_year, agent_j] = lt[c_year, agent_j].getValue()
            
            d[c_year, agent_j] = d[c_year, agent_j].getValue()
            ca[c_year, agent_j] = ca[c_year, agent_j].getValue()

            fa[c_year, agent_j] = fa[c_year, agent_j].getValue()
            gfi[c_year, agent_j] = gfi[c_year, agent_j].getValue()
            fni[c_year, agent_j] = fni[c_year, agent_j].getValue()
            e[c_year, agent_j] = e[c_year, agent_j].getValue()
            lr[c_year, agent_j] = ca[c_year, agent_j] / ( 1 / ML) * lt[c_year-1, agent_j]
            # sr[c_year, agent_j] = (lt[c_year, agent_j]) / (
            #     ca[c_year, agent_j] + fa[c_year, agent_j] - lt[c_year, agent_j]
            #     )
            u1[c_year, agent_j] = u1[c_year, agent_j].X
            u2[c_year, agent_j] = u2[c_year, agent_j].X

            # Calculate land production (€/ha)
            if A_va[c_year - 1, agent_j] == 0:
                A_va[c_year - 1, agent_j] = A_va_average

            u1_ha[c_year, agent_j] = u1[c_year, agent_j] / A_va[c_year - 1, agent_j]
            A_ha[c_year, agent_j] = A_ha[c_year - 1, agent_j] + u1_ha[c_year, agent_j]

            if A_ha[c_year, agent_j] != 0:
                A_va[c_year, agent_j] = a[c_year, agent_j] / A_ha[c_year, agent_j]
                prod_ha[c_year, agent_j] = gfi[c_year, agent_j] / A_ha[c_year, agent_j]
            else:
                A_va[c_year, agent_j] = 0
                prod_ha[c_year, agent_j] = 0

    if a[c_year, agent_j] == 0 and holder_retire == 2:
        print(f"Agent {agents_id[agent_j]} retierment")
        break


In [ ]:
# Optimization process - NO PREDICTION WINDOW#

# -- Simulate each year 
for c_year in range(1, sim_period):
    y = [0 for _ in range(nr_agents)]
    y1 = [0 for _ in range(nr_agents)]

    for agent_j in range(nr_agents):
        # ---- Equation ---- #
        a[c_year, agent_j] = a[c_year - 1, agent_j] + u1[c_year, agent_j] 
        fa[c_year, agent_j] = a[c_year, agent_j]                    # Total fixed assets

        lt[c_year, agent_j] = (1 - 1 / ML) * lt[c_year - 1, agent_j] + u2[c_year, agent_j]     # Long term liabilities

        gfi[c_year, agent_j] = PYLD[c_year] * a[c_year, agent_j]               # Gross Farm Income 
        
        fni[c_year, agent_j] = (
            gfi[c_year, agent_j]
            + SBT_c * a[c_year, agent_j]
            + SBT_nc
            - VAT
            - EFT
            - 1 / ML * lt[c_year - 1, agent_j]
            - u1[c_year, agent_j]
        )

        d[c_year, agent_j] = (1 + R) * d[c_year - 1, agent_j] + fni[c_year - 1, agent_j] - NFM * MEPI + u2[c_year, agent_j]

        # Currrent assets
        ca[c_year, agent_j] = d[c_year, agent_j]

        # Equity   
        e[c_year, agent_j] = fa[c_year, agent_j] + ca[c_year, agent_j] - lt[c_year, agent_j]

        # ---- Inequation - Constraints ---- #
        model.addConstr(d[c_year, agent_j] >= 0, name = "Deposits >= 0")
                                                    
        # ---- Parameters limits - Constraints ---- #                   
        model.addConstr(a[c_year, agent_j] >= 0, name = "Land >= 0")
            
        # -- Loan adquisition
        model.addConstr(u2[c_year, agent_j] >= 0, name = "Loans >= 0")                       # Loans cannot be negative
        model.addConstr(u2[c_year, agent_j] <= 2 * e[c_year - 1, agent_j], name = "Loans <= 2 Equity")            # Loans adquisition limitation

        # -- Land buy/sell expectations
        model.addConstr(u1[c_year, agent_j] <= d[c_year - 1, agent_j], name = "U1 (buy) <= deposits")      # It is not possible to invest in land buy above the deposits capacity    
        model.addConstr(u1[c_year, agent_j] >= -a[c_year - 1, agent_j], name = "U1 (sell) <= land")     # It is not possible to sell more land than available
        # model.addConstr(u1[c_year, agent_j] <= u2[c_year, agent_j]) 

        # Obj. function
        # y[agent_j] = fni[c_year, agent_j] / e[c_year - 1, agent_j] 
        y[agent_j] = ca[c_year, agent_j] - lt[c_year, agent_j]
        # y1[agent_j] = lt[c_year, agent_j]


    model.Params.OutputFlag = 0  # Set to 0 to suppress output during optimization

    # REMARK ---> model.setObjectiveN MINIMISE | Priority works inverse (higher number -> higher priority) | Weight works only if same priority
    # model.setObjectiveN(-y[0], index=0, priority=1, weight=0.3)
    # model.setObjectiveN(y1[0], index=1, priority=1, weight=0.9)

    model.setObjective(y[0], GRB.MAXIMIZE)

    model.update()
    model.optimize()

    # model.Status == 4 or model.Status == 3 or model.Status == 5
    if model.status == GRB.INFEASIBLE:
        print("Model is infeasible. Computing IIS...")
        model.computeIIS()
        model.write("model.ilp")

        print("\nThe following constraints are in the IIS:")
        for c in model.getConstrs():
            if c.IISConstr:
                print(f"{c.constrName}")

        print("Model INFEASIBLE. Attempting feasibility relaxation...")
        # relaxobjtype = 2  # 0: Minimize the weighted sum of relaxations. | 1: Minimize the weighted sum of squared relaxations. | 2: Minimize the weighted maximum of relaxations.
        # minrelax = True   # Minimize the relaxation
        # lbpen = False      # Allow relaxation of lower bounds
        # ubpen = True      # Allow relaxation of upper bounds
    
        model.feasRelaxS(0, True, False, True)
        model.optimize()

    if model.status == GRB.INFEASIBLE:
        print("Model is still infeasible.")
        break

    else:
        print("Model is feasible")
        # ---- Extract results ---- #
        for agent_j in range(nr_agents):
            a[c_year, agent_j] = a[c_year, agent_j].getValue()
            lt[c_year, agent_j] = lt[c_year, agent_j].getValue()
            
            d[c_year, agent_j] = d[c_year, agent_j].getValue()
            ca[c_year, agent_j] = ca[c_year, agent_j].getValue()

            fa[c_year, agent_j] = fa[c_year, agent_j].getValue()
            gfi[c_year, agent_j] = gfi[c_year, agent_j].getValue()
            fni[c_year, agent_j] = fni[c_year, agent_j].getValue()
            e[c_year, agent_j] = e[c_year, agent_j].getValue()
            lr[c_year, agent_j] = ca[c_year, agent_j] / ( 1 / ML) * lt[c_year-1, agent_j]
            sr[c_year, agent_j] = (lt[c_year, agent_j]) / (
                ca[c_year, agent_j] + fa[c_year, agent_j] - lt[c_year, agent_j]
                )
            u1[c_year, agent_j] = u1[c_year, agent_j].X
            u2[c_year, agent_j] = u2[c_year, agent_j].X
        
        if a[c_year, agent_j] == 0:
            print("Agents retierment")
            break


In [70]:
 # Storing the total results of the optimization for agent 
for v in range(c_year + 1):
    for j in range(nr_agents):
        a_f[v, j] = a[v, j]
        lt_f[v, j] = lt[v, j]
        ca_f[v, j] = ca[v, j]
        fa_f[v, j] = fa[v, j]
        d_f[v, j] = d[v, j]
        lr_f[v, j] = lr[v, j]
        sr_f[v, j] = sr[v, j]
        fni_f[v, j] = fni[v, j]
        gfi_f[v, j] = gfi[v, j]
        e_f[v, j] = e[v, j]
        np_f[v, j] = netp[v, j]
        u1_f[v, j] = u1[v, j]
        u2_f[v, j] = u2[v, j]

    year_f[v, j] = year + v

# Storing the total results of the optimization in a dataframe
agents_LP_optimisation = pd.DataFrame()
for agent_j in range(nr_agents):
    agent_LP_opt = pd.DataFrame()
    agent_LP_opt["Land"] = a_f[:, agent_j]
    agent_LP_opt["Loans"] = lt_f[:, agent_j]
    agent_LP_opt["Current Assets"] = ca_f[:, agent_j]
    agent_LP_opt["Fixed Assets"] = fa_f[:, agent_j]
    agent_LP_opt["Deposits"] = d_f[:, agent_j]
    agent_LP_opt["Liquidity Ratio"] = lr_f[:, agent_j]
    agent_LP_opt["Solvency Ratio"] = sr_f[:, agent_j]
    agent_LP_opt["Farm Net Income"] = fni_f[:, agent_j]
    agent_LP_opt["Gross Farm Income"] = gfi_f[:, agent_j]
    agent_LP_opt["Equity"] = e_f[:, agent_j]
    agent_LP_opt["Land buy+/sell-"] = u1_f[:, agent_j]
    agent_LP_opt["Loans acquisition"] = u2_f[:, agent_j]

    agent_LP_opt["agent_id"] = agents_id[agent_j]
    agent_LP_opt["Year"] = year_f[:, agent_j]

    agents_LP_optimisation = pd.concat([agents_LP_optimisation, agent_LP_opt])



In [ ]:
agents_LP_optimisation

In [119]:
productions = response_object.agriculturalProductions
agents = response_object.values
agent_policies = response_object.policyGroupRelations

In [ ]:
agent_policies

In [ ]:
def addRentOperations(dataset: pd.DataFrame, rentPrice: float = 520) -> List[LandRentJsonDTO]:
    # result: year, who_gives_land, to_whom, how_much
    result: List[LandRentJsonDTO] = []
    df_aziende = pd.read_csv('db/bilalncioce.csv', sep=';', decimal=',')
    df_aziende = df_aziende.astype({'Cod_Azienda':'string'})
    df_aziende = df_aziende[['Anno', 'Cod_Azienda', 'Affitti_Attivi','Affitti_Passivi']]
    # filter df_aziende to only have the rows where the pairs Anno, Cod_Azienda are in the parameter "dataset"
    # Create a multi-index in both dataframes
    df_aziende.set_index(['Cod_Azienda', 'Anno'], inplace=True)
    dataset.set_index(['Cod_Azienda', 'Anno'], inplace=True)
    # Filter df_aziende based on the multi-index in dataset
    df_aziende = df_aziende[df_aziende.index.isin(dataset.index)]
    df_aziende.reset_index(inplace=True)
    dataset.reset_index(inplace=True)
    
    years = df_aziende['Anno'].unique()
    for year in years:
        df_data = df_aziende[df_aziende['Anno'] == year]
        df_data = df_data[['Cod_Azienda', 'Affitti_Attivi','Affitti_Passivi']]
        rent_in = [ (x['Cod_Azienda'],x['Affitti_Passivi']/rentPrice) for i,x in df_data.iterrows() if x['Affitti_Passivi'] > 0.0]
        rent_out = [ (x['Cod_Azienda'],x['Affitti_Attivi']/rentPrice) for i,x in df_data.iterrows() if x['Affitti_Attivi'] > 0.0]
        rent_in_sorted = sorted(rent_in, key=lambda a: a[1])
        rent_out_sorted = sorted(rent_out, key = lambda a: a[1])
        while (len(rent_in_sorted) > 0 and len(rent_out_sorted) > 0):
            rent_in = rent_in_sorted.pop()
            rent_out = rent_out_sorted.pop()
            rent = min(rent_in[1], rent_out[1])
            rent_in = (rent_in[0], rent_in[1]- rent)
            rent_out = (rent_out[0], rent_out[1]- rent)
            result.append(LandRentJsonDTO(yearNumber=year, originFarmCode=rent_out[0], destinationFarmCode=rent_in[0], rentArea=rent, rentValue = rent*rentPrice))
            if rent_in[1] > 0:
                bisect.insort(rent_in_sorted, rent_in, key = lambda a: a[1])
            if rent_out[1] < 0:
                bisect.insort(rent_out_sorted, rent_out, key = lambda a: a[1])
        left_amount = sum([x[1] for x in rent_in_sorted if x[1] > 0.0]) + sum([x[1] for x in rent_out_sorted if x[1] > 0.0])
        print(f"not assigned rents for {left_amount} ha")
    return result

In [ ]:
import pandas as pd
import numpy as np

def buy_sell_match(buyers_c, sellers_c):
    # Sort buyers and sellers by their "Land buy+/sell-" values
    buyers_sorted = buyers_c.sort_values(by="Land buy+/sell-", ascending = False)
    sellers_sorted = sellers_c.sort_values(by="Land buy+/sell-")

    min_u1_diff = np.inf
    seller_id = buyer_id = None

    i, j = 0, 0
    while i < len(buyers_sorted) and j < len(sellers_sorted):
        buyer = buyers_sorted.iloc[i]
        seller = sellers_sorted.iloc[j]

        u1_diff = buyer["Land buy+/sell-"] + seller["Land buy+/sell-"]
        
        if abs(u1_diff) < min_u1_diff:
            min_u1_diff = abs(u1_diff)
            seller_id = seller["agent_id"]
            buyer_id = buyer["agent_id"]

        # Move the seller's pointer if u1_diff is positive, otherwise move the buyer's pointer
        if u1_diff > 0:
            j += 1
        else:
            i += 1

    return seller_id, buyer_id

# Example usage:
buyers_c = pd.DataFrame({'agent_id': [1, 2, 3], 'Land buy+/sell-': [10, 20, 1000]})
sellers_c = pd.DataFrame({'agent_id': [4, 5, 6], 'Land buy+/sell-': [-25, -15, -5]})

print(buy_sell_match(buyers_c, sellers_c))


In [ ]:
import pandas as pd
import numpy as np
from bisect import bisect_left

def buy_sell_match(buyers_c, sellers_c):
    # Sort buyers and sellers by their "Land buy+/sell-" values
    buyers_sorted = buyers_c.sort_values(by="Land buy+/sell-").reset_index(drop=True)
    sellers_sorted = sellers_c.sort_values(by="Land buy+/sell-").reset_index(drop=True)
    
    min_u1_diff = np.inf
    seller_id = buyer_id = None
    
    sellers_values = sellers_sorted["Land buy+/sell-"].values
    sellers_ids = sellers_sorted["agent_id"].values
    
    for _, buyer in buyers_sorted.iterrows():
        buyer_value = buyer["Land buy+/sell-"]
        buyer_id_temp = buyer["agent_id"]

        # Use binary search to find the closest seller value
        pos = bisect_left(sellers_values, -buyer_value)

        # Check the closest value on the left
        if pos > 0:
            left_seller_value = sellers_values[pos - 1]
            left_diff = abs(buyer_value + left_seller_value)
            if left_diff < min_u1_diff:
                min_u1_diff = left_diff
                seller_id = sellers_ids[pos - 1]
                buyer_id = buyer_id_temp

        # Check the closest value on the right (if within bounds)
        if pos < len(sellers_values):
            right_seller_value = sellers_values[pos]
            right_diff = abs(buyer_value + right_seller_value)
            if right_diff < min_u1_diff:
                min_u1_diff = right_diff
                seller_id = sellers_ids[pos]
                buyer_id = buyer_id_temp

    return seller_id, buyer_id

# Example usage:
buyers_c = pd.DataFrame({'agent_id': [1, 2, 3], 'Land buy+/sell-': [10, 20, 100]})
sellers_c = pd.DataFrame({'agent_id': [4, 5, 6], 'Land buy+/sell-': [-10, -15, -100]})

print(buy_sell_match(buyers_c, sellers_c))


In [ ]:
buyers_c.sort_values(by="Land buy+/sell-", ascending = False)

In [ ]:
sellers_c.sort_values(by="Land buy+/sell-")